In [1]:
from pyspark.sql import functions as F
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read all gold model_dataset parquet files built by main.py
md = spark.read.parquet("datamart/gold/model_dataset/gold_model_dataset_*.parquet")

print("Rows:", md.count(), "| Cols:", len(md.columns))
md.printSchema()

# Convert to pandas for EDA (safe since ~9k rows)
pdf = md.toPandas()
pdf["snapshot_date"] = pd.to_datetime(pdf["snapshot_date"])
pdf.head()

NameError: name 'spark' is not defined

In [2]:
# --- Imports & config ---------------------------------------------------------
import os, glob, json, re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve, confusion_matrix,
    classification_report
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

try:
    from xgboost import XGBClassifier
    XGB_AVAILABLE = True
except Exception:
    XGB_AVAILABLE = False

import joblib

# Plot settings
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

DATA_GLOB = "datamart/gold/model_dataset/gold_model_dataset_*.parquet"
ARTIFACT_DIR = Path("models")
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

def make_ohe():
    """Compat for scikit-learn versions: sparse_output (>=1.2) vs sparse (older)."""
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)


In [3]:
# --- Load ---------------------------------------------------------------------
paths = sorted(glob.glob(DATA_GLOB))
if not paths:
    raise FileNotFoundError(f"No files found for pattern: {DATA_GLOB}")

df_list = [pd.read_parquet(p) for p in paths]
df = pd.concat(df_list, ignore_index=True)

print("Rows:", len(df), "Cols:", df.shape[1])
display(df.head(3))

# Ensure snapshot_date as datetime
if "snapshot_date" in df.columns:
    df["snapshot_date"] = pd.to_datetime(df["snapshot_date"])
else:
    raise ValueError("Expected 'snapshot_date' column is missing.")

# Ensure target present and int
if "label" not in df.columns:
    raise ValueError("Expected 'label' target column is missing.")
df["label"] = df["label"].astype(int)


FileNotFoundError: No files found for pattern: datamart/gold/model_dataset/gold_model_dataset_*.parquet